In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split

import keras
from keras import Sequential
from keras.layers import Dense,Dropout
from keras import backend as K
from keras.constraints import Constraint

import tensorflow as tf

In [2]:
t = pd.read_csv('../input/lish-moa/train_features.csv')
tt = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

tt = tt[t["cp_type"]!="ctl_vehicle"]
t = t[t["cp_type"]!="ctl_vehicle"]

n_g = 772
n_c = 100

g_cols = ["g-"+str(i) for i in range(n_g)]
c_cols = ["c-"+str(i) for i in range(n_c)]

X = t[t.columns[4:]]
y = tt[tt.columns[1:]]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [3]:
model = Sequential()

model.add(Dense(200, input_dim=872, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(206, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

my_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode="min", restore_best_weights=True)]

model.fit(X_train,y_train,
          validation_data=(X_test, y_test),
          callbacks=my_callbacks,
          epochs=100)

Epoch 1/100
549/549 [==============================] - 2s 4ms/step - loss: 0.0482 - accuracy: 0.0411 - val_loss: 0.0218 - val_accuracy: 0.0606
Epoch 2/100
549/549 [==============================] - 2s 4ms/step - loss: 0.0230 - accuracy: 0.0732 - val_loss: 0.0205 - val_accuracy: 0.0920
Epoch 3/100
549/549 [==============================] - 2s 4ms/step - loss: 0.0213 - accuracy: 0.0878 - val_loss: 0.0203 - val_accuracy: 0.1007
Epoch 4/100
549/549 [==============================] - 2s 4ms/step - loss: 0.0202 - accuracy: 0.0976 - val_loss: 0.0206 - val_accuracy: 0.1098
Epoch 5/100
549/549 [==============================] - 2s 4ms/step - loss: 0.0195 - accuracy: 0.1010 - val_loss: 0.0205 - val_accuracy: 0.1089
Epoch 6/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0189 - accuracy: 0.1044 - val_loss: 0.0201 - val_accuracy: 0.1080
Epoch 7/100
549/549 [==============================] - 2s 4ms/step - loss: 0.0183 - accuracy: 0.1119 - val_loss: 0.0194 - val_accuracy: 0.1144

In [4]:
model.evaluate(X_train,y_train)
model.evaluate(X_test,y_test)

138/138 [==============================] - 0s 2ms/step - loss: 0.0171 - accuracy: 0.1344


[0.017056748270988464, 0.13439635932445526]

In [5]:
s = pd.read_csv('../input/lish-moa/test_features.csv')
sX = s[s.columns[4:]]

sy = model.predict(sX)

st = pd.DataFrame(data=sy, columns=y.columns)
st["sig_id"] = s["sig_id"]

st.loc[s["cp_type"]=="ctl_vehicle",y.columns] = 0

In [6]:
st.to_csv("submission.csv", index=False)